<a href="https://colab.research.google.com/github/hereagain-Y/TF_tutorial/blob/main/VAE_add_Normal_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

from six.moves import xrange
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np

from tqdm import tqdm
from torchvision.utils import save_image, make_grid
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split

Load CDR3 Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
s = []
my_file = open("/content/drive/My Drive/DL/VAE/NormalCDR3.txt", "r")
content_list = my_file.read().splitlines()
content_list=np.array(content_list)
print(content_list)


['CASSLKPNTEAFF' 'CASSAHIANYGYTF' 'CASSPRPNTEAFF' ... 'CASSVDVGYEQYF'
 'CASRLAGQETQYF' 'CASSVVPNTEAFF']


Load AA index data

In [11]:
d= {} # bipphysics dictionary
with open("/content/drive/My Drive/DL/VAE/AAidx_PCA.txt") as f:
    next(f)
    for line in f.readlines():
        line=line.strip().split('\t') #
        AA=line[0]
        tag=0
        values=[]
        for PC in line[1:]:
            values.append(float(PC))
        if tag==1: 
            continue
        d[AA]=values
 # AA


# normalization 

In [6]:
max(d['A'])
type(d['A'])

list

In [7]:
 
 #return 0 -1 (xi – min(x)) / (max(x) – min(x))

def normalize(d):
  result = {}
  for key, value in d.items():
    maxval = max(d[key])
    minval = min(d[key])
    newval=[]
    for number in value:

      newval.append((number-minval)/(maxval-minval))
    
    result[key] =newval
  return(result)
    



In [8]:

d_norm=normalize(d)
d_norm

{'A': [0.07525231330089575,
  0.11110993247250554,
  1.0,
  0.10085171430385075,
  0.3362586030378956,
  0.08589557098930195,
  0.3166716257763159,
  0.07845695355133557,
  0.29558950944630463,
  0.0,
  0.17264524667274148,
  0.02519795885608285,
  0.21399362066317887,
  0.20352855444366297,
  0.315091708147771],
 'C': [0.0,
  0.7495696809258403,
  0.07865597242639141,
  1.0,
  0.7627030301724574,
  0.8168259804237982,
  0.4333259554067372,
  0.16777058828085453,
  0.33203232126086285,
  0.5724424696175718,
  0.4816359939467978,
  0.39170914029908754,
  0.4294503690415155,
  0.37686420256824266,
  0.3449489815613006],
 'D': [1.0,
  0.25640988772162693,
  0.32592183369372396,
  0.4200305281353341,
  0.5744582792254195,
  0.1560420670881465,
  0.0,
  0.3091207506859722,
  0.258039340323857,
  0.2882727977487466,
  0.3895335207450628,
  0.38763175831637187,
  0.18676139620557258,
  0.5451097643479257,
  0.2696370438467982],
 'E': [1.0,
  0.0,
  0.8519057393700852,
  0.4383574918534277,
  

In [12]:
# max length
#comp_seq=data["amino_acid"].tolist()

comp_seq=content_list
print(comp_seq)
max_len=-1 

for AA in comp_seq:
    if len(AA)>max_len:
        max_len=len(AA)
        #res=AA
print(max_len )

['CASSLKPNTEAFF' 'CASSAHIANYGYTF' 'CASSPRPNTEAFF' ... 'CASSVDVGYEQYF'
 'CASRLAGQETQYF' 'CASSVVPNTEAFF']
19


In [13]:
PC_length=len(d_norm['C'])
def AAindexEncoding(Seq):
    length_seq=len(Seq)
    global max_len
    AAE=np.zeros([max_len,15])
    if length_seq<max_len:
        for amino in range(length_seq):
            AA=Seq[amino]# 
            AAE[amino,]=d_norm[AA] # add PC value 
            
        for amino in range(length_seq,max_len):
            AAE[amino,]=np.zeros(15)
    else: 
        for amino in range(length_seq): # zero padding
            AA=Seq[amino]# 
            AAE[amino,]=d_norm[AA]
        
    #AAE=np.transpose(AAE.astype(np.float32)) # row as PC. and column as AA sequence 
    return AAE 
  
def GetFeatures(file):
    #sequence=file['amino_acid'].tolist()
    #sequence=np.array(sequence)
    #sequence = file.read().splitlines()
    #sequence=np.array(sequence)
    hot_encode=[]
    for seq in file:
        hot_encode.append(AAindexEncoding(seq))
    hot_encode=np.array(hot_encode)
    result=np.array(hot_encode)
    return(result) # dimension: number of sequence [15*length(sequence)]


r1=GetFeatures(content_list)

In [14]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np

from tqdm import tqdm
from torchvision.utils import save_image, make_grid
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split

In [15]:
# transform 
kwargs = {'num_workers': 1, 'pin_memory': True} 
r1_transform=torch.from_numpy(r1) # from np to tensor
r1_transform=r1_transform.float()
r1_transform.shape
#r1_mean = torch.mean(r1_transform)
#r1_std = torch.std(r1_transform)
#r1_var =  torch.var(r1_transform)
#print("mean, std, var of tensor:\n", r1_mean, r1_std, r1_var)

# normalization 
#r1_transform = (r1_transform-r1_mean)/r1_var
#train_ds, test_ds = torch.utils.data.random_split(r2_transform, (30000, 10000))
train_ds, test_ds =torch.utils.data.random_split(r1_transform, (int(0.8*len(r1_transform)), len(r1_transform)-int(0.8*len(r1_transform))))
print(train_ds, test_ds)

train_loader = DataLoader(dataset=train_ds, batch_size=1000)
test_loader  = DataLoader(dataset=test_ds,  batch_size=1000)



<torch.utils.data.dataset.Subset object at 0x7f46aca30490> <torch.utils.data.dataset.Subset object at 0x7f46aca02d90>


In [16]:
# set paramters:
cuda = True
DEVICE = torch.device("cuda" if cuda else "cpu")


batch_size = 1000

x_dim=285 # 19*15
hidden_dim = 256
hidden_dim2 = 128

latent_dim = 64

lr = 1e-3

epochs = 200

In [17]:
class Encoder(nn.Module):
    
    def __init__(self, input_dim, hidden_dim,hidden_dim2, latent_dim):
        super(Encoder, self).__init__()

        self.FC_input = nn.Linear(input_dim, hidden_dim)
        self.FC_input2 = nn.Linear(hidden_dim, hidden_dim)
        self.FC_input3 = nn.Linear(hidden_dim, hidden_dim2)
        self.FC_mean  = nn.Linear(hidden_dim2, latent_dim)
        self.FC_var   = nn.Linear (hidden_dim2, latent_dim)
        
        self.LeakyReLU = nn.LeakyReLU(0.2)
        
        self.training = True
        
    def forward(self, x):
        h_       = self.LeakyReLU(self.FC_input(x))
        h_       = self.LeakyReLU(self.FC_input2(h_))
        h_       = self.LeakyReLU(self.FC_input3(h_))
        mean     = self.FC_mean(h_)
        log_var  = self.FC_var(h_)                     # encoder produces mean and log of variance 
                                                       #             (i.e., parateters of simple tractable normal distribution "q"
        
        return mean, log_var

In [18]:
class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim,hidden_dim2, output_dim):
        super(Decoder, self).__init__()
        self.FC_hidden = nn.Linear(latent_dim, hidden_dim2)
        self.FC_hidden3 = nn.Linear(hidden_dim2, hidden_dim)
        self.FC_hidden2 = nn.Linear(hidden_dim, hidden_dim)
        self.FC_output = nn.Linear(hidden_dim, output_dim)
        
        self.LeakyReLU = nn.LeakyReLU(0.2)
        
    def forward(self, x):
        h     = self.LeakyReLU(self.FC_hidden(x))
        h     = self.LeakyReLU(self.FC_hidden3(h))
        h     = self.LeakyReLU(self.FC_hidden2(h))
        
        x_hat = torch.sigmoid(self.FC_output(h))
        return x_hat

In [19]:
class Model(nn.Module):
    def __init__(self, Encoder, Decoder):
        super(Model, self).__init__()
        self.Encoder = Encoder
        self.Decoder = Decoder
        
    def reparameterization(self, mean, var):
        epsilon = torch.randn_like(var).to(DEVICE)        # sampling epsilon        
        z = mean + var*epsilon                          # reparameterization trick
        return z
        
                
    def forward(self, x):
        mean, log_var = self.Encoder(x)
        z = self.reparameterization(mean, torch.exp(0.5 * log_var)) # takes exponential function (log var -> var) ]
        x_hat            = self.Decoder(z)
        
        return x_hat, mean, log_var

In [20]:
encoder = Encoder(input_dim=x_dim, hidden_dim=hidden_dim, hidden_dim2=hidden_dim2, latent_dim=latent_dim)
decoder = Decoder(latent_dim=latent_dim, hidden_dim = hidden_dim, hidden_dim2=hidden_dim2,output_dim = x_dim)

model = Model(Encoder=encoder, Decoder=decoder).to(DEVICE)

In [21]:
from torch.optim import Adam

BCE_loss = nn.BCELoss()

def loss_function(x, x_hat, mean, log_var):
    #reproduction_loss = nn.functional.binary_cross_entropy(x_hat, x, reduction='sum')
    # try MSE 
    MSELoss_criterion = nn.MSELoss()
    reproduction_loss = MSELoss_criterion(x_hat, x) 
    KLD      = - 0.5 * torch.sum(1+ log_var - mean.pow(2) - log_var.exp())

    return reproduction_loss + KLD


optimizer = Adam(model.parameters(), lr=lr)

In [22]:
print("Start training VAE...")
model.train()

for epoch in range(epochs):
    overall_loss = 0
    overall_testloss= 0
    train_loss= []
    testtoal_loss= []
    for batch_idx, x in enumerate(train_loader):
        x = x.view(len(x), x_dim)
        x = x.to(DEVICE)

        optimizer.zero_grad()

        x_hat, mean, log_var = model(x)
        loss = loss_function(x, x_hat, mean, log_var)
        overall_loss += loss.item()
        train_loss.append(loss.item())
        
        loss.backward()
       
        optimizer.step()
      #test loss
    for batch_idx, x in enumerate(test_loader):
        x = x.view(len(x), x_dim)
        x = x.to(DEVICE)

        #optimizer.zero_grad()

        pred, mean, log_var = model(x)
        test_loss = loss_function(x, pred, mean, log_var)
        overall_testloss += test_loss.item()
        testtoal_loss.append(test_loss.item())

    if (epoch % 10 == 0):
      #print('====> Epoch %d done! Average Loss:  = %.2e, Average test loss = %.2e' % (epoch,overall_loss / (batch_idx*batch_size),overall_testloss/(batch_idx*batch_size)))
        
        
      print("\tEpoch", epoch , "complete!", "\tAverage Loss: ", overall_loss / (batch_idx*batch_size),
        "\tAverage Test Loss: " , overall_testloss/(batch_idx*batch_size))
    
print("Finish!!")

Start training VAE...
	Epoch 0 complete! 	Average Loss:  0.05449788778168815 	Average Test Loss:  0.0006113174557685852
	Epoch 10 complete! 	Average Loss:  0.0003543674232704299 	Average Test Loss:  8.8515433881964e-05
	Epoch 20 complete! 	Average Loss:  0.0002926900147327355 	Average Test Loss:  7.167203884039607e-05
	Epoch 30 complete! 	Average Loss:  0.0002649787435574191 	Average Test Loss:  6.852815353444645e-05
	Epoch 40 complete! 	Average Loss:  0.00026267068726675854 	Average Test Loss:  6.428837137562888e-05
	Epoch 50 complete! 	Average Loss:  0.00025533879761184964 	Average Test Loss:  6.332914584449359e-05
	Epoch 60 complete! 	Average Loss:  0.0002601987128811223 	Average Test Loss:  6.21769694345338e-05
	Epoch 70 complete! 	Average Loss:  0.00035656349520598137 	Average Test Loss:  6.281075413737979e-05
	Epoch 80 complete! 	Average Loss:  0.00025882551978741376 	Average Test Loss:  6.246963994843619e-05
	Epoch 90 complete! 	Average Loss:  0.00024379046261310578 	Average Tes

In [23]:


#save model
torch.save(model, '/content/drive/My Drive/DL/VAE/VAE_modify_model_2000_echo.apx')

model = torch.load('/content/drive/My Drive/DL/VAE/VAE_modify_model_2000_echo.apx')

In [24]:
# load model. 

my_file2 = open("/content/drive/My Drive/DL/VAE/NormalCDR3_test.txt", "r")
content_list2 = my_file2.read().splitlines()

#content_list2=np.array(content_list2)

neww=[x for x in content_list2 if len(x)<=19]


neww=np.array(neww)


r2=GetFeatures(neww)

r2.shape

(19851, 19, 15)

numpy.ndarray

In [25]:
#

# normalization 
#r2_transform = (r2_transform-r2_mean)/r2_var
#r2_transform.shape
#result = model(r2_transform)
#m2 = r2.view(len(r2), 285)
r2_transform=torch.from_numpy(r2).float() # change to tensor and float 
m2=r2_transform.view(len(r2_transform),285) # flatten 
result = model(m2.cuda())[0] # 

result2=result.cpu().view( len(result),19,15 ).detach().numpy()


In [26]:
result2

array([[[0.00166776, 0.7532813 , 0.07518455, ..., 0.4257379 ,
         0.37116298, 0.34017888],
        [0.08733966, 0.11833699, 0.9908606 , ..., 0.2117609 ,
         0.20132579, 0.31000584],
        [0.96998054, 0.6697961 , 0.40851477, ..., 0.2931407 ,
         0.27959305, 0.27749228],
        ...,
        [0.00180132, 0.00840301, 0.00641774, ..., 0.00874487,
         0.00694305, 0.00585583],
        [0.00139423, 0.00263938, 0.00238904, ..., 0.00235901,
         0.00212453, 0.00246961],
        [0.00172117, 0.00234231, 0.00170169, ..., 0.00208615,
         0.00170741, 0.00213096]],

       [[0.00173521, 0.751322  , 0.0770069 , ..., 0.42656243,
         0.37128526, 0.3393428 ],
        [0.08813239, 0.12049115, 0.99064016, ..., 0.21363084,
         0.20330606, 0.31091   ],
        [0.9695841 , 0.6688101 , 0.40919048, ..., 0.29399082,
         0.2800664 , 0.27867675],
        ...,
        [0.00188848, 0.0087174 , 0.00671104, ..., 0.00899589,
         0.0072574 , 0.00608097],
        [0.0

In [ ]:
result2.shape

(19851, 19, 15)

Save array for later use 

In [ ]:
np.save('/content/drive/My Drive/DL/VAE/validate_array', result2)

In [ ]:
# save the normalized dictionary 
import pickle 


with open("/content/drive/My Drive/DL/VAE/normalize_pca.txt", "wb") as myFile:
    pickle.dump(d_norm, myFile)


In [ ]:
# example to load

with open("/content/drive/My Drive/DL/VAE/normalize_pca.txt", "rb") as myFile:
    newPull= pickle.load(myFile)

print(newPull)

{'A': [0.07525231330089575, 0.11110993247250554, 1.0, 0.10085171430385075, 0.3362586030378956, 0.08589557098930195, 0.3166716257763159, 0.07845695355133557, 0.29558950944630463, 0.0, 0.17264524667274148, 0.02519795885608285, 0.21399362066317887, 0.20352855444366297, 0.315091708147771], 'R': [1.0, 0.0, 0.4038784221987408, 0.5654030339035435, 0.19358241833934803, 0.9218071081013374, 0.706684933100523, 0.386763003529879, 0.4859773194257129, 0.35364725786412415, 0.853611611157344, 0.6896935049070329, 0.5301256891869882, 0.5887127624896037, 0.6865122676206136], 'N': [1.0, 0.33875792059923404, 0.08051659481278953, 0.5409271544498805, 0.17196565841462272, 0.08031021902130163, 0.18874324146258997, 0.5291298621814148, 0.3042291274231407, 0.46516048086757544, 0.3558216551514157, 0.38102091917816505, 0.07103751169386222, 0.0, 0.4581217601781321], 'D': [1.0, 0.25640988772162693, 0.32592183369372396, 0.4200305281353341, 0.5744582792254195, 0.1560420670881465, 0.0, 0.3091207506859722, 0.258039340323